In [1]:
import pandas as pd
import torch

In [ ]:
with open('mh.txt') as f:
    data = f.read()
    print(data)

In [2]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [10]:
from collections import defaultdict
import spacy
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import Chroma
import random
import en_core_web_trf
from pprint import pprint
from lmqg import TransformersQG
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

class MCQGeneration:
    def __init__(self, file):
        # Initialize model and embeddings
        self.model = TransformersQG(model="lmqg/t5-base-squad-qag")
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.persons = []

        # Load SpaCy model (ensure it's installed)
        try:
            self.nlp = spacy.load("en_core_web_trf")
        except OSError:
            print("Downloading SpaCy model 'en_core_web_trf'...")
            from spacy.cli import download
            download("en_core_web_trf")
            self.nlp = spacy.load("en_core_web_trf")

        # Read input file
        with open(file, "r") as f:
            self.data = f.read()

    def ner_extraction(self):
        """Extract named entities and generate documents."""
        doc = self.nlp(self.data)
        words = defaultdict(set)

        # Extract entities from sentences
        for sent in doc.sents:
            for ent in sent.ents:
                words[ent.text].add((ent.text, sent.text, ent.label_))

        context_answers = [item for value in words.values() for item in value]
        persons = [(i[1], i[0], i[2]) for i in context_answers]

        # Create documents with metadata
        documents = [
            Document(page_content=context, metadata={"ner_tag": ner_tag, "class": label})
            for context, ner_tag, label in persons
        ]
        return documents, persons

    def search(self, documents, persons):
        """Perform similarity search and generate MCQs."""
        docsearch = Chroma.from_documents(documents, self.embeddings)
        print("Documents ingested successfully!")

        for doc in documents:
            query = doc.page_content
            results = docsearch.similarity_search(query)
            linked = {}
            question_answer = self.model.generate_qa(query)

            # Use a temporary dictionary to avoid modifying during iteration
            temp_linked = {}
            for i in question_answer:
                linked[i] = i
                temp_linked[i] = i

            for keys, values in temp_linked.items():
                query = values[0]
                ner = values[1]

                # Filter persons based on NER tag
                x = [j[2] for j in persons if j[1] == ner]

                results = docsearch.similarity_search(query, k=100)  # Retrieve top-100 results
                options = []

                for result in results:
                    print(result.metadata.get("class"))
                    if result.metadata.get("class") in x:
                        options.append(result.metadata.get("ner_tag"))

                # Handle cases where fewer than 3 options are available
                if len(options) < 3:
                    options += [None] * (3 - len(options))  # Pad with None if not enough options

                if len(options) >= 3:  # Ensure there are enough options to sample from
                    random_values = random.sample(options, 3)
                    random_values.append(question_answer[0][1])  # Add the correct answer
                    random.shuffle(random_values)  # Shuffle the options
                    linked["options"] = random_values

        yield linked





In [ ]:
mcq = MCQGeneration('mh.txt')

In [12]:
documents,persons = mcq.ner_extraction()

In [ ]:
all_results = []
h = 0

# Consume the generator and collect results
for linked in mcq.search(documents, persons):  # Ensure `mcq` is defined and initialized
    if h >= 1:  # Stop collecting after 10 outputs
        break
    all_results.append(linked)  # Append each yielded result to the list
    h += 1

# Print the results to verify
print(all_results)


In [ ]:
import spacy
import en_core_web_trf
from collections import defaultdict

nlp = en_core_web_trf.load()

# Load the pre-trained model
nlp = spacy.load('en_core_web_trf')
doc = nlp(data)
words = defaultdict(set)
for sent in doc.sents:
    for ent in sent.ents:
        words[ent.label_].add((ent.text,sent.text,ent.label_)) 
        print(ent.text, ent.label_)

In [ ]:
words.keys()

In [6]:
context_answers = []
for key, value in words.items():
    for i in words[key]:
        context_answers.append(i)
persons = [(i[1],i[0],i[2]) for i in context_answers]

In [ ]:
context_answers

In [ ]:
[i[2] for i in persons if i[1] == 'Santhal Pargana' for x in i]

In [11]:
from langchain.schema import Document

# Example: persons is a list of tuples [(context, ner_tag), ...]


# Convert to Document objects
documents = [
    Document(page_content=context, metadata={"ner_tag": ner_tag,"class":label})
    for context, ner_tag, label in persons
]


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
# Initialize embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docsearch = Chroma.from_documents(documents, embeddings)

print("Documents ingested successfully!")


In [ ]:
persons[1][0]

In [ ]:
# Perform a similarity search
query = 'The burning of foreign cloth by Gandhi and the burning of Manusmriti by Ambedkar are not to be seen as mere acts of sentiment. Rather, foreign cloth and Manusmriti represented the bondage and slavery for India. So too, a pinch of salt from the ocean, and a drop of water from the Mahad tank were acts of political catharsis and social philosophy.'
results = docsearch.similarity_search(query, k=50)  # Retrieve top-1 result

for result in results:
    print("Matched Context:", result.page_content)
    print("NER Tag:", result.metadata.get("ner_tag"))
    print("NER label:", result.metadata.get("class"))

doc = text

In [15]:
example = ('the Santhal Revolt',
               'It was after the Santhal Revolt of 1855-56 that the Santhal Pargana was created out of the districts of Bhagalpur and Birbhum.')

In [128]:
context = '''Sher Singh was sent to suppress the revolt, but he himself joined Mulraj, leading to a mass uprising  in Multan.'''
entity = example[0]

In [ ]:
from pprint import pprint
from lmqg import TransformersQG
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document

# initialize model
model = TransformersQG(model='lmqg/t5-base-squad-qag') # or TransformersQG(model='lmqg/t5-base-squad-qg-ae') 
# paragraph to generate pairs of question and answer


In [ ]:
question_answer = model.generate_qa(context)
# the output is a list of tuple (question, answer)


In [ ]:
question_answer

In [131]:
linked = []


In [132]:
solution = {}
for i in question_answer:
    linked.append(i)
    



In [ ]:
linked


In [ ]:
for i in linked:
    print(i)

In [ ]:
persons

In [ ]:
all = []
for i in linked:
    query=(i[0])
    print(query)
    ner = (i[1])
    results = docsearch.similarity_search(query, k=5)  # Retrieve top-1 result
    x = [j[2] for j in persons if j[1] == ner]
    print(x)
    options = []
    for result in results:
        if result.metadata.get("class") in x and result.metadata.get("ner_tag") not in options:
            options.append(result.metadata.get("ner_tag"))
    all.append((query,ner,options))

In [ ]:
all

In [ ]:
valid = []
for i in all:
    if i[2] != []:
        valid.append(i)
valid

In [ ]:
import json
import random
output = []
for question, correct_answer, options in valid:
    # Remove the correct answer from options
    filtered_options = [opt for opt in options if opt != correct_answer]
    
    # Randomly select 3 incorrect answers
    random_choices = random.sample(filtered_options, 3)
    
    # Add the correct answer and shuffle
    random_choices.append(correct_answer)
    random.shuffle(random_choices)
    
    # Append formatted question set to output
    output.append({
        "question": question,
        "correctAnswer": correct_answer,
        "options": random_choices
    })

# Write to JSON file
with open("questions.json", "w") as f:
    json.dump(output, f, indent=4)

print(output)


In [ ]:
solution

In [ ]:
linked
    

In [145]:
r = {}

In [148]:
r[1] = 'x'

In [162]:
r[2] = 'y'

In [ ]:
for keys,values in r.items():
    print(values)

In [ ]:
for key,values in r.items():
    for i in range(len(values)):
        h = values
        print(h)

In [ ]:
print(r.keys)